In [1]:
import pandas as pd
import elasticsearch
from elasticsearch_dsl import Search, A, Q
from elasticsearch_dsl.query import MatchAll, Match

/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


multiterm aggregation
#https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-terms-aggregation.html

In [2]:
es = elasticsearch.Elasticsearch(hosts=[{'host': 'localhost', 'port': 9200}])

In [3]:
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": "AUTO",
                        "fields": [
                            "phrase",#,#,
                            "phrase.ext"
                            #"phrase.bigrams"
                            #"phrase.ngram"
                        ]
                    }
                },
            ],
            "should": [
                {
                    "multi_match": {
                        "query": "",
                        "fuzziness": 1,
                        "fields": [
                            "norm_phrase"
                            #"norm_phrase.bigrams"
                            #"norm_phrase.ngram"
                        ]
                    }
                }
            ]
        }
    },
    "size": 25,
    "sort": {
        "_score": {
            "order": "desc"
        }
    },
}

query = {
    "query": {
        "match_all": {}
    }
}
res = es.search(index="rdrs110722", body = query)

/tmp/ipykernel_11825/3176541370.py:46: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="rdrs110722", body = query)


In [163]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")

In [156]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw")

In [164]:
response = s.execute()

In [165]:
listed = []
#for hit in s[0:3800]:
for hit in s.scan():
    listed.append(hit.to_dict())

In [148]:
res = s.aggs.bucket("UniqueDrugnames", agg_type="terms", field="Drugname_kw").bucket("UniqueDrugnames", agg_type="sum", field="Diseasename_kw"


In [170]:
s = Search(using=es, index="rdrs110722").source(["Drugname_kw", "Diseasename_kw", "review_url_kw"]).query("match_all")
s.aggs.bucket("drugs", A("cardinality", field="Drugname_kw"))
res = s.execute()

RequestError: RequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [Drugname_kw] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')

In [123]:
s["UniqueDrugnames"].bucket("tags_per_category")

TypeError: '<' not supported between instances of 'str' and 'int'

In [166]:
pd.DataFrame(listed)

,Drugname_kw,Diseasename_kw
0,[Амиксин],"[герпеса, Герпес]"
1,"[Иммунофлазид, Иммунофлазид]","[ГРВИ, грипа]"
2,"[Гавискон, Гавискон, Гавискон]",NaN
3,[Феназепам],NaN
4,[Виферон],[простуд]
...,...,...
3816,"[Кленбутерол, кленбутерол]",NaN
3817,"[Леденцы Вулкан Лакричные, леденцах лакричных,...",NaN
3818,"[Диафлекс РОМФАРМ, Диафлекс, омепрозол, Диафле...",NaN
3819,"[Целебрекс, цереблекс]",[обострении радикулита]


# простой индекс

In [13]:

#s = Search(using=es, index="rdrs110722_scheme2").query("match_all")
#s = Search(using=es, index="rdrs110722_scheme2").query("match", Indications="кашель, насморк")
s = Search(using=es, index="med800k_scheme2").query("match_all").count()
s = Search(using=es, index="med800k_scheme2").query("match", Indications="кашель, насморк").count()
query = Q("match", Drugnames='мукалтин') | Q("match", Indications='кашель, насморк')
s = Search(using=es, index="med800k_scheme2").query(query).sort({'Review_count': {"order": "desc"}})

In [8]:
s = Search(using=es, index="med800k_scheme2").query("match_all").count()

In [5]:
s

524878

In [9]:
s

519049

In [213]:
res = s.execute()
res

AttributeError: 'int' object has no attribute 'execute'

In [14]:
listed = []
for hit in s[0:1000]:
#for hit in s.scan():
    listed.append(hit.to_dict())

In [255]:
listed

[{'Drugnames': 'називин',
  'Diseasenames': '',
  'Indications': 'насморк',
  'ADRs': '',
  'ADR_reviews_count': 0,
  'Negated_ADE_reviews_count': 4,
  'Neg_reviews_count': 4,
  'Pos_reviews_countg': 10,
  'Review_count': 10,
  'Review_urls': ['https://otzovik.com/review_3247191.html',
   'https://otzovik.com/review_2588017.html',
   'https://otzovik.com/review_1225711.html',
   'https://otzovik.com/review_3998878.html',
   'https://otzovik.com/review_1637031.html',
   'https://otzovik.com/review_1138675.html',
   'https://otzovik.com/review_863539.html',
   'https://otzovik.com/review_482834.html',
   'https://otzovik.com/review_3796096.html',
   'https://otzovik.com/review_6400143.html']},
 {'Drugnames': 'мукалтин',
  'Diseasenames': '',
  'Indications': 'кашель, кашля',
  'ADRs': '',
  'ADR_reviews_count': 0,
  'Negated_ADE_reviews_count': 0,
  'Neg_reviews_count': 0,
  'Pos_reviews_countg': 10,
  'Review_count': 10,
  'Review_urls': ['https://otzovik.com/review_5851979.html',
   'h

In [15]:
pd.DataFrame(listed)

,Drugnames,Diseasenames,Indications,ADRs,ADR_reviews_count,Negated_ADE_reviews_count,Neg_reviews_count,Pos_reviews_countg,Review_count,Review_urls
0,називины,,насморк,,0,4,4,10,10,"[https://otzovik.com/review_3247191.html, http..."
1,мукалтины,,"кашель, кашля",,0,0,0,10,10,"[https://otzovik.com/review_5851979.html, http..."
2,эреспал,,кашель,,0,5,5,7,8,"[https://otzovik.com/review_208624.html, https..."
3,протаргол,,насморк,,0,2,2,5,8,"[https://otzovik.com/review_2425481.html, http..."
4,амброксол,,"кашель, кашля",,0,0,0,7,8,"[https://otzovik.com/review_1497325.html, http..."
...,...,...,...,...,...,...,...,...,...,...
995,"отривино, отривины с ментол",,"гнусавить, заложен нос, заложенность носа, нас...",,0,0,0,1,1,[https://otzovik.com/review_4428641.html]
996,"аквамарисы, физраствор, чистонос",,"насморк, сопли",,0,0,0,1,1,[https://otzovik.com/review_2248059.html]
997,"аквалор, аквамарисы",,"заложенность носа, кашель, кашля, сопли",непрерывно на коленях сползти на пол,1,0,1,1,1,[https://otzovik.com/review_7093286.html]
998,"амбробен, эреспал",,"кашель, кашель сухой",,0,1,1,1,1,[https://otzovik.com/review_2470332.html]


In [193]:
import json
with open("/s/ls4/users/grartem/SAG_MED/med-corpus-analyser/data/raw/ner_800k_jsonl/ner_800k.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        print(data)
        break

{'text_id': 0, 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.', 'entities': {'0': {'entity_id': 0, 'text': 'очень понравился кожу', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 277, 'end': 293}]}, '1': {'entity_id': 1, 'text': 'хорошо увлажняет и питает', 'tag': ['Disease:DisTypeBNE-Pos'], 'spans': [{'begin': 373, 'end': 398}]}, '2

In [191]:
data

{'text_id': 0,
 'text': 'Я люблю пользоваться всякими гелями для душа, особенно, которые приятно пахнут. Кремом-гелем пользовалась впервые. Его нам подарили на работе на 8 марта. Что могу сказать-отличный подарок. Раньше продукцией Dove я практически не пользовалась, разве только мылом, крем-гель мне очень понравился. Нежный, пенящийся, приятный легкий аромат. Не сушит кожу после использования, хорошо увлажняет и питает. Очень вкусно пахнет кокосом и жасмином, ненавязчиво. Пользоваться-сплошное наслаждение. Теперь я проверила его действие на себе и смело могу купить в подарок близким людям и друзьям. Могу порекомендовать крем-гель Dove всем любительницам понежеться в ванне, я думаю, что вам он тоже понравится.',
 'entities': {'0': {'entity_id': 0,
   'text': 'очень понравился кожу',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 277, 'end': 293}]},
  '1': {'entity_id': 1,
   'text': 'хорошо увлажняет и питает',
   'tag': ['Disease:DisTypeBNE-Pos'],
   'spans': [{'begin': 

In [11]:
import requests
data = {
    "Indications": ["кашель"]
}
r = requests.post("http://127.0.0.1:5000/scheme2_request", data = data)
r.text

'[\n  {\n    "ADR_reviews_count": 0,\n    "ADRs": "",\n    "Diseasenames": "",\n    "Drugnames": "\\u043c\\u0443\\u043a\\u0430\\u043b\\u0442\\u0438\\u043d",\n    "Indications": "\\u043a\\u0430\\u0448\\u0435\\u043b\\u044c, \\u043a\\u0430\\u0448\\u043b\\u044f",\n    "Neg_reviews_count": 0,\n    "Negated_ADE_reviews_count": 0,\n    "Pos_reviews_countg": 10,\n    "Review_count": 10,\n    "Review_urls": [\n      "https://otzovik.com/review_5851979.html",\n      "https://otzovik.com/review_4635686.html",\n      "https://otzovik.com/review_3351632.html",\n      "https://otzovik.com/review_7031087.html",\n      "https://otzovik.com/review_2965205.html",\n      "https://otzovik.com/review_7261591.html",\n      "https://otzovik.com/review_807520.html",\n      "https://otzovik.com/review_3393842.html",\n      "https://otzovik.com/review_3577013.html",\n      "https://otzovik.com/review_3662989.html"\n    ]\n  },\n  {\n    "ADR_reviews_count": 0,\n    "ADRs": "",\n    "Diseasenames": "",\n    "Dru

In [16]:
import pandas as pd
pd.DataFrame(r.json())

,ADR_reviews_count,ADRs,Diseasenames,Drugnames,Indications,Neg_reviews_count,Negated_ADE_reviews_count,Pos_reviews_countg,Review_count,Review_urls
0,0,,,мукалтин,"кашель, кашля",0,0,10,10,"[https://otzovik.com/review_5851979.html, http..."
1,0,,,амброксол,"кашель, кашля",0,0,7,8,"[https://otzovik.com/review_1497325.html, http..."
2,0,,,проспан,"кашель, кашля",3,3,5,8,"[https://otzovik.com/review_949903.html, https..."
3,0,,,эреспал,кашель,4,4,6,7,"[https://otzovik.com/review_208624.html, https..."
4,0,,,амбробене,кашель,0,0,6,7,"[https://otzovik.com/review_2453656.html, http..."
...,...,...,...,...,...,...,...,...,...,...
995,1,"Нарушение вкуса, Отдышка, Расстройство желудка...","ангина, ангиной, бронхитом","амброксол, муколтином","боли в горле, болит, кашель, кашля, кашля в гр...",1,0,1,1,[https://otzovik.com/review_7054832.html]
996,0,,астмы,"амбробене, амброгексал, амброксол","кашель, кашлять, кашляю бронхов, мокроты, остр...",0,0,1,1,[https://otzovik.com/review_2054292.html]
997,0,,"обострившийся бронхит, простуды",но - шпа,"в, горле казалось прошлись колючками, кашель, ...",0,0,1,1,[https://otzovik.com/review_422626.html]
998,0,,,синекод,"кашель, мокрота, сухого кашля, сухой , царапаю...",1,1,0,1,[https://otzovik.com/review_3035603.html]
